In [1]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import math
from tagger import Tagger
from sentence_transformers import SentenceTransformer
import umap.umap_ as umap

data = pd.read_csv("../data/train_valid_all.csv")

OPENAI API KEY OBTAINED


/Users/kaylahuang/opt/anaconda3/envs/mercor/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Extract the responses and labels
X = data['Response']
y = data['Label']  # Assuming there is a 'Label' column for classification
y = y - 1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_test), len(y_test))

# Convert the text data to tf-idf features
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)
y_pred_rf = rf_classifier.predict(X_test_tfidf)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {rf_accuracy}")

# Train an XGBoost classifier
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(X_train_tfidf, y_train)
y_pred_xgb = xgb_classifier.predict(X_test_tfidf)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {xgb_accuracy}")


136 136


Random Forest Accuracy: 0.6102941176470589


/Users/kaylahuang/opt/anaconda3/envs/mercor/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [09:36:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.6397058823529411


In [20]:
y_test_df = pd.DataFrame(y_test)
y_test_df["xgb_guess"] = y_pred_xgb + 1
y_test_df["length"] = X_test.apply(len).values

In [22]:
y_test_df

,Label,xgb_guess,length
646,1,2,1104
336,0,1,40
63,1,2,1027
367,2,2,1516
101,1,1,103
...,...,...,...
110,0,1,57
82,1,2,1226
51,1,2,1115
617,1,3,1841


In [25]:
from sklearn.linear_model import LogisticRegression

# Prepare the data for logistic regression
X_log_reg = y_test_df.drop(columns=['Label'])
y_log_reg = y_test_df['Label']

# Train a logistic regression model
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_log_reg, y_log_reg)

# Predict using the logistic regression model
y_pred_log_reg = log_reg.predict(X_log_reg)

# Calculate accuracy
log_reg_accuracy = accuracy_score(y_log_reg, y_pred_log_reg)
print(f"Logistic Regression Accuracy: {log_reg_accuracy}")


Logistic Regression Accuracy: 0.6617647058823529


/Users/kaylahuang/opt/anaconda3/envs/mercor/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
